In [8]:
import pandas as pd

In [11]:
df = pd.read_csv('supervised_learning_data/40_lightbulbs_16_goals_100_trajectories_time__20250817-115001.csv', index_col= False )

In [12]:
df.columns

Index(['rnn_hidden_state', 'bayes_posterior', 'trajectory_id', 'time_step'], dtype='object')

In [30]:
lens = df.groupby('trajectory_id').size()
lens.max(), lens.mean(), lens.describe()

(np.int64(891),
 np.float64(27.9),
 count    100.00000
 mean      27.90000
 std       87.89037
 min        8.00000
 25%       13.75000
 50%       17.00000
 75%       21.00000
 max      891.00000
 dtype: float64)

In [ ]:
# 2) Recompute via max(time_step) + 1 and compare
lens1 = df.groupby('trajectory_id').size()
lens2 = df.groupby('trajectory_id')['time_step'].max().add(1)
float(lens1.mean()), float(lens2.mean()), (lens1 == lens2).all()


(27.9, 27.9, np.True_)